# ARR Matching Process

This notebook outlines several cells on how to use the helper modules for several steps of the ARR Matching.

To use this notebook, please place this notebook at the **root** of `openreview-scripts`. To check, ensure that the path `./arr/matching` resolves to the folder containing the helper modules.

## Setup
Initialize your clients, an `ARR` object so you can write code parameterized by `venue` which generalizes across cycles, an `SACACMatching` object to run the SAC and AC matchings and an `ARRMatcher` object for helper code

In [ ]:
from arr.matching.sac_core import SACACMatching
from arr.matching.core import ARRMatcher
import openreview, time

request_form_id = ''

client_v1 = openreview.Client(
    username='',
    password='',
    baseurl='https://devapi.openreview.net'
)
client = openreview.api.OpenReviewClient(
    username='',
    password='',
    baseurl='https://devapi2.openreview.net'
)
venue = openreview.helpers.get_conference(client_v1, request_form_id)
matching = SACACMatching(
    client_v1, client, request_form_id,
    matcher_baseurl='https://devapi2.openreview.net'
)
general_matcher = ARRMatcher(
    client,
    request_form_id
)

## Pre-Matching
The cells below outline some steps that occur before any matchings should be run. Typically this looks like:
1. Running the ARR-specific scripts to setup the resubmission data and calculate the initial values for research areas
2. Registering authors as one role or another
3. Force updating the loads of specific reviewers or ACs
4. Moving reviewers or ACs between groups depending on where extra capacity is needed

### ARR-Specific Scripts
The scripts are named:

1. Setup_Reviewer_Matching
2. Setup_AE_Matching
3. Setup_SAE_Matching

You can either do this by:

(1) Running these scripts in the backend asynchronously and checking below for the logs:

`https://api2.openreview.net/logs/process?invitation=aclweb.org/ACL/ARR/2099/January/-/Setup_AE_Matching`

or for the dev site:

`https://devapi2.openreview.net/logs/process?invitation=aclweb.org/ACL/ARR/2099/January/-/Setup_AE_Matching`

**NOTE**: There are timeout limits on these processes, if it fails, you may have to run this locally on your machine



In [ ]:
client.post_invitation_edit(
    invitations=venue.get_meta_invitation_id(),
    readers=[venue.id],
    writers=[venue.id],
    signatures=[venue.id],
    invitation=openreview.api.Invitation(
        id = f"{venue.id}/-/Setup_AE_Matching",
        content = {
            'count': {'value': 1}
        }
    )
)

(2) Running this in your machine using the helper module.

Another way to run this in your machine is to copy-and-paste the whole [function](https://github.com/openreview/openreview-py/blob/master/openreview/arr/management/setup_reviewer_matching.py) into a notebook or script and structure the code as:
```python
def process(client, invitation):
    # body goes here...
client = openreview.api.OpenReviewClient(...)
invitation = client.get_invitation(f"{venue_id}/-/Setup_Reviewer_Matching")
process(client, invitation)
```

In [ ]:
general_matcher.setup_reviewer_matching_data()
general_matcher.setup_ac_matching_data()
general_matcher.setup_sac_matching_data()

### Registering Authors

Since the ARR editors don't have access to post data on behalf of the authors, registering an author just means adding them to the group. The helper code in this repository will search for the latest track information we have for the users using the data in the cycle.

**NOTE**: Any time you register authors to a role, you should provide an `author_to_load` dictionary so that they are made available. It is recommended to store this information somewhere on disk, and load the data within the notebook so you have a record of this information and which author is registered to which role

In [ ]:
general_matcher.register_authors_as_reviewers(
    author_tilde_ids=[
        '~Author_User53',
        '~Author_User47'
    ],
    author_to_load={}
)

### Make Available
These functions synchronize the user loads with their notes and the "make available" part is similar to above - passing in a dictionary that maps profile IDs to their desired load

In [ ]:
general_matcher.make_reviewers_available(
    reviewer_to_load={
        '~Author_User53': 10,
        '~Author_User47': 10
    }
)

### Transfer Between Roles
These functions don't delete or post any data, because of the same restrictions as mentioned above. Instead, it just moves users between roles.

Because it doesn't delete or post data, the track and load information will still belong to the original role unless the user submits a new form. Since we use the latest note, if they submit a maximum load request to their new role, it will take this into account

In [ ]:
general_matcher.transfer_reviewers_to_acs(
    reviewer_tilde_ids=[
        '~Author_User53',
        '~Author_User47'
    ]
)
general_matcher.transfer_acs_to_reviewers(
    ac_tilde_ids=[
        '~Author_User53',
        '~Author_User47'
    ]
)

## SAC and AC Matchings
These cells below outline the function to use to run the SAC and AC matchings. They also outline how to reset the data in order to recompute the matching.

Many of these calls have guardrails to prevent you from accidentally posting or deleting data. If you are sure of the workflow, you can pass `force=True` to certain function calls.

However, you shouldn't be afraid of messing up the matching data for these stages before the assignments are deployed, as these changes are opaque to any of the users before deployment

### Clearing All Data

This cell will load all information on the current AC and SAC matchings and delete the data for a clean slate

In [ ]:
RESET_SAC_AC_DATA = False
import time

if RESET_SAC_AC_DATA:
    sac_configuration_notes = client.get_all_notes(
        invitation=f"{venue.get_senior_area_chairs_id()}/-/Assignment_Configuration",
    )
    ac_configuration_notes = client.get_all_notes(
        invitation=f"{venue.get_area_chairs_id()}/-/Assignment_Configuration",
    )
    for note in ac_configuration_notes + sac_configuration_notes:
        role = note.invitations[0].split('/-/')[0]
        title = note.content['title']['value']
        prop_asm_count = client.get_edges_count(
            invitation=f"{role}/-/Proposed_Assignment",
            label=title
        )
        agg_score_count = client.get_edges_count(
            invitation=f"{role}/-/Aggregate_Score",
            label=title
        )
        print(f"{role} {title} ({prop_asm_count} proposed assignments, {agg_score_count} aggregate scores)")

        if prop_asm_count > 0:
            client.delete_edges(
                invitation=f"{role}/-/Proposed_Assignment",
                label=title,
                wait_to_finish=True
            )
        if agg_score_count > 0:
            client.delete_edges(
                invitation=f"{role}/-/Aggregate_Score",
                label=title,
                wait_to_finish=True
            )
        
        if note.ddate is None:
            client.post_note_edit(
                invitation=venue.get_meta_invitation_id(),
                readers=[venue.id],
                writers=[venue.id],
                signatures=[venue.id],
                note=openreview.api.Note(
                    id=note.id,
                    ddate=int(time.time() * 1000)
                )
            )

### Resetting All Data
This cell will recompute data that gets recomputed during the SAC and AC matching. You should run this to reset the states of the following data:

1. AC Research Area Edges
2. AC Conflicts
3. SAC Research Area

In [ ]:
# Uncomment this line to recompute the SAC affinity scores
# NOTE: This call can take an hour or two but it is required when adding new SACs
# general_matcher.compute_sac_affinity_scores()

# Uncomment this line to recompute the AC affinity scores
# NOTE: This call can take several hours but it is required when adding new ACs
# general_matcher.compute_ac_affinity_scores()

In [ ]:
general_matcher.sync_ac_tracks()
general_matcher.sync_sac_tracks()
general_matcher.compute_ac_conflicts()

### Running the SAC and AC Matchings
This single function runs the code contained in the `arr/matching/sac_*.py` files

In [ ]:
matching.run_matching(
    ac_title='ac-matching',   # <-- title of the final AC matching
    sac_title='sac-matching', # <-- title of the final SAC matching
)

## Reviewer Matchings
These cells below outline the function to setup/reset the reviewer matching data and run the reviewer matchings

### Setting up/Recomputing the Reviewer Data
You can run this cell to reset and synchronize the reviewer data before running a matching. You only need to do this if you have added new reviewers to the pool

In [ ]:
# Uncomment this line to recompute the reviewer affinity scores
# NOTE: This call can take several hours but it is required when adding new reviewers
# general_matcher.compute_reviewer_affinity_scores()

In [ ]:
loaded_forced_loads = {}
general_matcher.compute_reviewer_conflicts()
general_matcher.sync_reviewer_tracks()
general_matcher.sync_reviewer_loads(
    forced_loads=loaded_forced_loads
)

### Running the Reviewer Matching

After performing the setup (affinity scores, conflicts, tracks anad loads), you can run the matching from the assignments page. You can go to a previous cycle's reviewer matching, click "Copy" on the deployed assignments parameters and use this as a template.

If it's easier, you cna also go to `constants.py` and reference `DEFAULT_REVIEWER_MATCHING_CONFIG`

## Post-Matching

These cells are typically operations you would run either after the assignments are computed or after they are deployed

### Sanity Checking

You can call `run_sanity_checks()` to run the a suite of sanity checks which includes:
1. Checks ACs and Reviewers have their Custom_Max_Papers and Maximum_Load_And_Unavailability notes synced (this will return non-zero for any one whose load was forced to be a certain number)
2. Checks for any papers that are missing SAC/AC/Reviewer assignments
3. Checks that no one is assigned more papers than their Custom_Max_Papers indicates
4. Checks that the SAC-AC mapping exists (for each ACs assigned to an SAC. the AC is only assigned papers where the SAC is the mapped SAC)
5. Checks for any conflicts
6. Checks for any role overlap at the group-level (this one can be run individually before assignments are computed or deployed)

In [ ]:
r = general_matcher.run_sanity_checks()

### Fill in Missing Assignments

Sometimes you may need to bulk remove assignments and want to fill them in with automated assignments without running the whole matcher. The `recommend_reviewers` and `recommend_acs` functions use the following logic to find new reviewers/ACs:

1. Retrieve submissions and assignment data
2. Retrieve conflicts, affinity scores, research area edges, custom max paper edges and submitted load notes
3. For each paper:
4. Iterate over all users and skip if they are (1) already assigned, (2) alreaday invited or (3) in conflict
5. Compute their load (automatically assigned + any assignments made during this process) and skip if it exceeds their Custom_Max_Papers
6. If they pass these checks, they are marked as available. All avaiable users have their aggregate scores calclulated and selected the highest aggregate score user

In [ ]:
print(
    general_matcher.recommend_acs(
        num_required_assignments=1,
        ac_assignment_title='ac-matching',
        dry_run=False
    )
)